In [1]:
! git clone https://github.com/vonum/multi-prophet.git

Cloning into 'multi-prophet'...
remote: Enumerating objects: 398, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 398 (delta 49), reused 58 (delta 23), pack-reused 304
Receiving objects: 100% (398/398), 5.97 MiB | 778.00 KiB/s, done.
Resolving deltas: 100% (208/208), done.


In [2]:
!pip install pystan==2.19.1.1
!pip install fbprophet
import sys
sys.path.append('multi-prophet')
from multi_prophet import MultiProphet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 430.1 kB/s eta 0:00:0000:0100:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 346.5 kB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 836.3 kB/s eta 0:00:00 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for fbprophet: filename=fbprophet-0.7.1-py3-none-any.whl size=4838784 sha256=17fff5bd2306e98e19914a1d4cf610968acf919fe6c7529a00fcfc7bdb46a1ed
  Stored in directory: /root/.cache/pip/wheels/cd/a1/12/db63ff624de492fe6cccf676091a0860fdde2ffde4bc3280e2
Successfully built fbprophet
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 1.1.0
    Uninstalling cmdstanpy-1.1.0:
      Successfully uninstalled cmdstanpy-1.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [3]:
import os
import logging
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from prophet import Prophet

In [5]:
df = pd.read_csv('monica_weather_2011-2016.csv')
df["date"] = pd.to_datetime(df["de-date"], format='%d.%m.%Y')

In [12]:
df = pd.read_csv('../wofost/reanalysis_2010_2015.csv')
df["date"] = pd.to_datetime(df["DAY"], format='%Y%m%d')

In [13]:
df = df.rename(columns={"date": "ds"})

In [14]:
m = MultiProphet(columns=['IRRAD', 'TMIN', 'TMAX', 'VAP', 'WIND', 'RAIN'])
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In [7]:
m = MultiProphet(columns=["tavg", "tmin", "tmax", "wind", "globrad", "precip", "relhumid"])
m.fit(df)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [15]:
future = m.make_future_dataframe(periods=366, include_history=False)
forecast = m.predict(future)

In [9]:
test = pd.read_csv('reanalysis_2017.csv')

In [16]:
test = pd.read_csv('../wofost/reanalysis_2016.csv')

In [17]:
test['IRRAD'] = forecast['IRRAD'].yhat.to_numpy()
test['TMIN'] = forecast['TMIN'].yhat.to_numpy()
test['TMAX'] = forecast['TMAX'].yhat.to_numpy()
test['VAP'] = forecast['VAP'].yhat.to_numpy()
test['WIND'] = forecast['WIND'].yhat.to_numpy()
test['RAIN'] = forecast['RAIN'].yhat.to_numpy()

In [10]:
test['tavg'] = forecast['tavg'].yhat.to_numpy()
test['tmin'] = forecast['tmin'].yhat.to_numpy()
test['tmax'] = forecast['tmax'].yhat.to_numpy()
test['wind'] = forecast['wind'].yhat.to_numpy()
test['globrad'] = forecast['globrad'].yhat.to_numpy()
test['precip'] = forecast['precip'].yhat.to_numpy()
test['relhumid'] = forecast['relhumid'].yhat.to_numpy()

In [18]:
test.to_csv('../wofost/multi_prophet_2016.csv', index=False)